# Tanzu Build Service - Installation Script

## vSphere Installation (HomeLab)

- PKS Setup on vSphere - UAA with OIDC Enabled
- Harbor Setup with latest version or above 1.10+
- Installaing MetalLB - LB for bare metal and home labs
- Ingress Controller Setup - Contour


### PKS Setup with OIDC enabled

Setup Required:

- PKS version: 1.6.1-build.6
- Harbor Version: Version v1.10.1-f3e11715
- OIDC Enabled: PKS OpsMan -> Enterprise PKS -> UAA Configuration -> Configure created clusters to use UAA as the OIDC provider. - Enable 

### UAA User Creation

In [ ]:
!uaac target https://api.pks.lab.digitalkloud.com:8443 --ca-cert /var/tempest/workspaces/default/root_ca_certificate

#### Authenticate with UAA by running following command: uaac token client get admin -s ADMIN-CLIENT-SECRET (Get it from OpsMan Credentials Tab)

In [ ]:
!uaac token client get admin -s <PKS ADMIN CLIENT SECRET>

#### Create a new User

In [ ]:
!uaac user add <username> --emails <user@email> -p <password>

In [ ]:
!uaac member add pks.clusters.admin <username>

### Create a PKS cluster with new user

In [ ]:
!pks login -u shashmi -p Admin@1234 -a api.pks.lab.digitalkloud.com -k

In [2]:
!pks plans


Name   ID                                    Description
Small  8A0E21A8-8072-4D80-B365-D1F502085560  This plan will configure a lightweight Kubernetes cluster. Not recommended for production workloads.



In [86]:
!pks create-cluster lab-cluster --external-hostname sh.lab.cluster.digitalkloud.com --plan small


PKS Version:              1.6.1-build.6
Name:                     lab-cluster
K8s Version:              1.15.5
Plan Name:                Small
UUID:                     7495bdfb-6a7b-4d62-9a73-09a55cd00c18
Last Action:              CREATE
Last Action State:        in progress
Last Action Description:  Creating cluster
Kubernetes Master Host:   sh.lab.cluster.digitalkloud.com
Kubernetes Master Port:   8443
Worker Nodes:             3
Kubernetes Master IP(s):  In Progress
Network Profile Name:     

Use 'pks cluster lab-cluster' to monitor the state of your cluster



In [4]:
!pks clusters


PKS Version    Name           k8s Version  Plan Name  UUID                                  Status     Action
1.6.1-build.6  lab01-cluster  1.15.5       Small      7039eae2-c482-452a-9cfc-ac5f817b338c  succeeded  CREATE



In [ ]:
!pks get credentials

In [ ]:
!kubectl config use-context lab01-cluster

In [ ]:
!kubectl get nodes

In [ ]:
!kubectl get po

### Installing MetalLB on PKS Cluster

- Reference Link for Installation: Install metallb - https://metallb.universe.tf/
- Copy the apply command and run it
- Create a config map and updated the internal IP range with the Node IP ranges in your env
- Test by deploying an nginx app

In [87]:
!kubectl apply -f https://raw.githubusercontent.com/google/metallb/v0.8.3/manifests/metallb.yaml

namespace/metallb-system created
podsecuritypolicy.policy/speaker created
serviceaccount/controller created
serviceaccount/speaker created
clusterrole.rbac.authorization.k8s.io/metallb-system:controller created
clusterrole.rbac.authorization.k8s.io/metallb-system:speaker created
role.rbac.authorization.k8s.io/config-watcher created
clusterrolebinding.rbac.authorization.k8s.io/metallb-system:controller created
clusterrolebinding.rbac.authorization.k8s.io/metallb-system:speaker created
rolebinding.rbac.authorization.k8s.io/config-watcher created
daemonset.apps/speaker created
deployment.apps/controller created


In [6]:
!kubectl get no -o wide

NAME                                   STATUS   ROLES    AGE   VERSION   INTERNAL-IP      EXTERNAL-IP      OS-IMAGE             KERNEL-VERSION      CONTAINER-RUNTIME
1709af9a-4208-4149-b8e4-4e191c0e0c6a   Ready    <none>   14h   v1.15.5   192.168.40.204   192.168.40.204   Ubuntu 16.04.6 LTS   4.15.0-66-generic   docker://18.9.9
76612da5-ced1-4bbb-8bf2-9882e8ac672d   Ready    <none>   14h   v1.15.5   192.168.40.203   192.168.40.203   Ubuntu 16.04.6 LTS   4.15.0-66-generic   docker://18.9.9
eb2de809-6472-4328-8fd2-84d4f675555a   Ready    <none>   14h   v1.15.5   192.168.40.202   192.168.40.202   Ubuntu 16.04.6 LTS   4.15.0-66-generic   docker://18.9.9


#### Create a ConfigMap:

- Identify the IP ranges from the Internal IP range above and select a range
- Apply the below config map and verify it that it lies in the same namespace
- Deploy a NGINX image to see if LoadBalance type works

apiVersion: v1
kind: ConfigMap
metadata:
  namespace: metallb-system
  name: config
data:
  config: |
    address-pools:
    - name: default
      protocol: layer2
      addresses:
      - 192.168.40.238-192.168.40.250



In [88]:
!kubectl describe configmaps -n metallb-system

Name:         config
Namespace:    metallb-system
Labels:       <none>
Annotations:  kubectl.kubernetes.io/last-applied-configuration:
                {"apiVersion":"v1","data":{"config":"address-pools:\n- name: default\n  protocol: layer2\n  addresses:\n  - 192.168.40.238-192.168.40.250\...

Data
====
config:
----
address-pools:
- name: default
  protocol: layer2
  addresses:
  - 192.168.40.238-192.168.40.250

Events:  <none>


In [8]:
!kubectl run nginx --image nginx

kubectl run --generator=deployment/apps.v1 is DEPRECATED and will be removed in a future version. Use kubectl run --generator=run-pod/v1 or kubectl create instead.
deployment.apps/nginx created


In [9]:
!kubectl expose deploy nginx --port 80 --type LoadBalancer

service/nginx exposed


In [10]:
!kubectl get all

NAME                         READY   STATUS    RESTARTS   AGE
pod/nginx-7bb7cd8db5-btb44   1/1     Running   0          17s

NAME                 TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)        AGE
service/kubernetes   ClusterIP      10.100.200.1     <none>           443/TCP        14h
service/nginx        LoadBalancer   10.100.200.215   192.168.40.240   80:32633/TCP   15s

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx   1/1     1            1           17s

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-7bb7cd8db5   1         1         1       17s


Open a browser and navigate to: http://192.168.40.240 - You will be able to see NGINX home page.

![NGINX](img/nginx-server.png)

Clean up the nginx deployment

In [11]:
!kubectl delete deployment nginx

deployment.extensions "nginx" deleted


In [14]:
!kubectl delete services/nginx

service "nginx" deleted


In [15]:
!kubectl get all

NAME                 TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.100.200.1   <none>        443/TCP   14h


### Install Contour - Ingress Controller

In [97]:
!kubectl apply -f https://projectcontour.io/quickstart/contour.yaml

namespace/projectcontour created
serviceaccount/contour created
configmap/contour created
customresourcedefinition.apiextensions.k8s.io/ingressroutes.contour.heptio.com configured
customresourcedefinition.apiextensions.k8s.io/tlscertificatedelegations.contour.heptio.com configured
customresourcedefinition.apiextensions.k8s.io/httpproxies.projectcontour.io configured
customresourcedefinition.apiextensions.k8s.io/tlscertificatedelegations.projectcontour.io configured
serviceaccount/contour-certgen created
rolebinding.rbac.authorization.k8s.io/contour created
role.rbac.authorization.k8s.io/contour-certgen created
job.batch/contour-certgen created
clusterrolebinding.rbac.authorization.k8s.io/contour unchanged
clusterrole.rbac.authorization.k8s.io/contour unchanged
role.rbac.authorization.k8s.io/contour-leaderelection created
rolebinding.rbac.authorization.k8s.io/contour-leaderelection created
service/contour created
service/envoy created
deployment.apps/contour created
daemonset.apps/envoy

#### Test a sample application with Contour Ingress Controller

In [81]:
!kubectl apply -f https://projectcontour.io/examples/kuard.yaml

deployment.apps/kuard created
service/kuard created
ingress.networking.k8s.io/kuard unchanged


#### Retrieve the external address of Contour’s Envoy load balancer

In [90]:
!kubectl get -n projectcontour service envoy -o wide

NAME    TYPE           CLUSTER-IP      EXTERNAL-IP      PORT(S)                      AGE   SELECTOR
envoy   LoadBalancer   10.100.200.32   192.168.40.238   80:31955/TCP,443:32139/TCP   64s   app=envoy


In [22]:
!kubectl delete deployment/kuard

deployment.extensions "kuard" deleted


### Create Persistent Volume and Claim for TBS

Persistent Volume:

apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: fast
  annotations:
    storageclass.kubernetes.io/is-default-class: "true"
provisioner: kubernetes.io/vsphere-volume
parameters:
  datastore: datastore1
  diskformat: thin
  fstype: ext3


In [24]:
!pwd

/Users/shashmi/workspace/greatlakes-repo/tanzu-jupyter-scripts/TBS


In [29]:
!kubectl apply -f setup/tbs-pv.yaml

storageclass.storage.k8s.io/fast created


In [34]:
!kubectl apply -f setup/tbs-pcv.yaml

persistentvolumeclaim/pvcsc001 created


In [91]:
!kubectl get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM              STORAGECLASS   REASON   AGE
pvc-d0b4e653-2b3b-4884-a435-63e81134d8c2   2Gi        RWO            Delete           Bound    default/pvcsc001   fast                    20s


In [92]:
!kubectl get pvc

NAME       STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
pvcsc001   Bound    pvc-d0b4e653-2b3b-4884-a435-63e81134d8c2   2Gi        RWO            fast           24s


### Configuring Certificates for TBS

#### Copy Harbor SSL cert and CA cert from OpsMan Harbor tile and Add it to OCX Store

- Reference Link: https://support.securly.com/hc/en-us/articles/206058318-How-to-install-the-Securly-SSL-certificate-on-Mac-OSX-

#### Install duffle and pb CLI

- Download Duffle and PB cli from pivnet
- rename duffle-0.0.4-darwin to duffle
- rename pb-0.0.4-darwin to pb
- chmod +x duffle
- chmod +x pb
- mv duffle /usr/local/bin
- mv pb /usr/local/bin

In [83]:
!duffle version

0.3.2-beta.1


In [84]:
!pb version

CLI Version: 0.0.4 (d5c17831)

ERROR: please run `pb api set` to configure an API target


In [52]:
ls

README.md                               img/
Tanzu Build Service Installation.ipynb  setup/


Give the **path** where you have downloaded build-service-0.0.4.tgz or run it from that location

In [ ]:
!duffle relocate -f ./build-service-0.0.4.tgz -m ./relocated.json -p harbor.lab.digitalkloud.com/buildservice

Once you run the above command, **relocated.json** will be created and a builder package will be created inside your Harbor Repository under **buildservice** project

![HARBOR-Buildservice](img/harbor-buildservice.png)

#### Create a build-service-parameters.json

{
   "ingress_annotations":{
      "kubernetes.io/ingress.class":"contour"
   }
}


#### Create a DNS for your TBS

- For example: pbs.lab.digitalkloud.com
- Use openssl if you have to generate a self signed certificate with the below command
- Provide all the information including FQDN when prompted

In [ ]:
!openssl req -x509 -newkey rsa:4096 -keyout key.pem -out cert.pem -days 365

#### Create a build-service-credentials.yml file with below content:

- Reference Link : https://docs.pivotal.io/build-service/0-0-4/installing.html


name: build-service-credentials
credentials:
 - name: kube_config
   source:
     path: "PATH-TO-KUBECONFIG"
   destination:
     path: "/root/.kube/config"
 - name: ca_cert #This is your Harbor CA Cert
   source:
     path: "PATH-TO-CA"
   destination:
     path: "/cnab/app/cert/ca.crt"
 - name: tls_cert
   source:
     path: "PATH-TO-TLS-CERTIFICATE"
   destination:
     path: "/cnab/app/cert/tls.crt"
 - name: tls_key
   source:
     path: "PATH-TO-TLS-PRIVATE-KEY"
   destination:
     path: "/cnab/app/cert/tls.key"

In [71]:
!openssl x509 -subject -issuer -noout -in cert.pem

Can't open cert.pem for reading, No such file or directory
140735746089856:error:02001002:system library:fopen:No such file or directory:crypto/bio/bss_file.c:69:fopen('cert.pem','r')
140735746089856:error:2006D080:BIO routines:BIO_new_file:no such file:crypto/bio/bss_file.c:76:
unable to load certificate


### Install Build Service

In [57]:
ls ../../../TANZU/tanzu-build-service/

HarborCerts-Lab/                cert.pem
build-service-0.0.4.tgz         key.pem
build-service-credentials.yml   relocated.json
build-service-parameteres.json


#### As above in the directory above where you have the following:

- build-service-credentials.yml
- build-service-parameters.json
- buildserver tar file (build-service.0.0.4.tgz)
- relocated.json (generated in the above steps)

Fill out the remaining as per your environment and run the duffle command:


duffle install pbs -c ./build-service-credentials.yml  \
    --set domain=pbs.lab.digitalkloud.com \
    --set kubernetes_env=platform-services \
    --set docker_registry=harbor.lab.digitalkloud.com \
    --set registry_username="admin" \
    --set registry_password="<password>" \
    --set uaa_url="https://api.pks.lab.digitalkloud.com:8443" \
    -p ./build-service-parameteres.json \
    -f ./build-service-0.0.4.tgz \
    -m ./relocated.json


Avoiding duffle error: claim already exists
If an install fails or if you prefer to reset your Kubernetes cluster rather than uninstalling, you can remove existing duffle claims by deleting the files in ~/.duffle/claims. This will allow you to re-install without encountering the “claim already exists” error.



In [94]:
!kubectl get ns

NAME                    STATUS   AGE
build-service-gateway   Active   5h15m
default                 Active   5h58m
kpack                   Active   5h15m
kube-node-lease         Active   5h58m
kube-public             Active   5h58m
kube-system             Active   5h58m
metallb-system          Active   5h33m
pks-system              Active   5h45m
projectcontour          Active   5h30m


#### Create a pbs user from UAA

In [ ]:
!uaac token client get admin -s <PKS Adming Client Secret>

In [ ]:
!uaac user add pbs -p pbs --emails pbs@pivotal.io

In [66]:
!uaac users | grep pbs

NOTE: Gem::Specification#rubyforge_project= is deprecated with no replacement. It will be removed on or after 2019-12-01.
Gem::Specification#rubyforge_project= called from /Library/Ruby/Gems/2.3.0/specifications/cf-uaa-lib-3.13.0.gemspec:16.
NOTE: Gem::Specification#rubyforge_project= is deprecated with no replacement. It will be removed on or after 2019-12-01.
Gem::Specification#rubyforge_project= called from /Library/Ruby/Gems/2.3.0/specifications/highline-1.7.10.gemspec:20.
NOTE: Gem::Specification#rubyforge_project= is deprecated with no replacement. It will be removed on or after 2019-12-01.
Gem::Specification#rubyforge_project= called from /Library/Ruby/Gems/2.3.0/specifications/em-http-request-1.1.5.gemspec:16.
NOTE: Gem::Specification#rubyforge_project= is deprecated with no replacement. It will be removed on or after 2019-12-01.
Gem::Specification#rubyforge_project= called from /Library/Ruby/Gems/2.3.0/specifications/em-socksify-0.3.2.gemspec:16.
NOTE: Gem::Specification#rubyf

In [95]:
!kubectl get services --all-namespaces

NAMESPACE               NAME                    TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)                      AGE
build-service-gateway   build-service-gateway   ClusterIP      10.100.200.113   <none>           80/TCP                       5h15m
default                 kubernetes              ClusterIP      10.100.200.1     <none>           443/TCP                      5h58m
kpack                   kpack-webhook           ClusterIP      10.100.200.24    <none>           443/TCP                      5h15m
kube-system             kube-dns                ClusterIP      10.100.200.2     <none>           53/UDP,53/TCP                5h45m
kube-system             kubernetes-dashboard    NodePort       10.100.200.51    <none>           443:32034/TCP                5h45m
kube-system             metrics-server          ClusterIP      10.100.200.72    <none>           443/TCP                      5h45m
pks-system              fluent-bit              ClusterIP      10.100.200.10  

### Verify PBS Installation

#### Target the Build Service installation by running: pb api set BUILD-SERVICE-DOMAIN

In [106]:
!pb api set https://pbs.lab.digitalkloud.com --skip-ssl-validation


ERROR: Failed to set Build Service API endpoint: failed to connect to endpoint: https://pbs.lab.digitalkloud.com/v1/info: Get https://pbs.lab.digitalkloud.com/v1/info: dial tcp 192.168.40.238:443: connect: connection refused

Usage:
  pb api set <api-url> [flags]

Examples:
pb api set https://build-service.pivotal.io

Flags:
  -h, --help                  help for set
      --skip-ssl-validation   skip ssl validation for https connections




In [103]:
ls

README.md                               img/
Tanzu Build Service Installation.ipynb  setup/


### Test PBS Service